In [ ]:
## Data Manipulation
import pandas as pd
import numpy as np

## Scraping
import requests

## Plotting
import matplotlib.dates as mdates
import matplotlib.pyplot as plt
import seaborn as sns

## Spatial Manipulation
import geopandas as gpd
from shapely.geometry import Point

## Mapping
import folium
from folium import plugins

## Misc
import functools
from itertools import product
import contextily as ctx

pd.options.display.max_columns = None
from matplotlib.lines import Line2D 

from scipy.stats.stats import pearsonr

In [ ]:
df = pd.read_csv("pm_no_clean.csv", index_col=0)

In [ ]:
df = df.drop_duplicates(subset=['pod_id_location'])

In [ ]:
#turning df into GIS file
geometry = [Point(xy) for xy in zip(df.Longitude, df.Latitude)]
crs = {'init': 'epsg:4326'}
gdf = gpd.GeoDataFrame(df, crs = crs, geometry=geometry)
gdf = gdf.to_crs({'init': 'epsg:3857'})

In [ ]:
#loading gis road network
path = "/Users/oliverpaul/Data_Science/EDF/LAEI GIS - traffic/SHP/LAEI2016_AADT_VKM_toid.shp"
roads = gpd.read_file(path)
roads['VKMTOTAL16'] = roads['VKMTOTAL16'].str.replace(',', '')
roads['VKMTOTAL16'] = roads['VKMTOTAL16'].replace('-', np.nan)

In [ ]:
roads['VKMTOTAL16'] = pd.to_numeric(roads['VKMTOTAL16'], errors='coerce')
roads = roads.dropna(subset=['VKMTOTAL16'])
roads['VKMTOTAL16'] = roads['VKMTOTAL16'].astype(int)

In [ ]:
#filtering roads dataset for only major roads - motorway or A Road
roads = roads[(roads['DESC_TERM'] == "Motorway") | (roads['DESC_TERM'] == "A Road")]

In [ ]:
roads = roads.to_crs({'init': 'epsg:3857'})

In [ ]:
#loading ULEZ
path = "/Users/oliverpaul/Data_Science/EDF/LAEI - GIS geographies/Ultra_Low_Emissions_Zone.gpkg"
ULEZ = gpd.read_file(path)
crs = {'init': 'epsg:27700'}
ULEZ = gpd.GeoDataFrame(ULEZ, crs=crs)
ULEZ = ULEZ.to_crs({'init': 'epsg:3857'})

In [ ]:
gdf['tot_avg_no2'] = gdf['tot_avg_no2'].fillna(0)

In [ ]:
gdf['tot_avg_no2'] = gdf['tot_avg_no2'].astype(int)

In [ ]:
plt.rcParams['figure.dpi'] = 300

ax = ULEZ.plot(figsize=(20, 20), edgecolor='k', facecolor='none', linewidth=2, zorder = 1)

roads[(roads['DESC_TERM'] == "A Road") & ((roads['VKMTOTAL16'] > 100000) & (roads['VKMTOTAL16'] < 1000000))].plot(ax=ax, 
                                                                               column='VKMTOTAL16', 
                                                                               legend=False, 
                                                                               cmap='Reds', 
                                                                               linewidth = 3, zorder = 5)

gdf.plot(ax=ax, markersize = gdf['tot_avg_no2']**2, zorder = 50, alpha=0.5)
gdf.plot(ax=ax, markersize = 20, zorder = 50, facecolor='black')

ax.set(xlim=(-20000, -5800), ylim=(6706000, 6717500))
ctx.add_basemap(ax=ax, url=ctx.providers.Stamen.TonerLite)

circ1 = Line2D([0], [0], linestyle="none", marker="o", markersize=25, markerfacecolor="lightgray", alpha = 0.85)
circ2 = Line2D([0], [0], linestyle="none", marker="o", markersize=12, markerfacecolor="lightgray", alpha = 0.85)

leg = ax.legend((circ1, circ2), ("Higher Average", "Lower Average"), 
                      numpoints=1, 
                      loc=4,
                      fontsize ='x-large')

ax.set_title('Relative A-Road traffic flow intensity and relative average No2 readings for ULEZ area\n', 
                 fontdict={'fontsize': '20', 'fontweight' : '2'})

plt.axis('off')
plt.show()

### One of the highest reading sensors in is Bank. The fact that this sensor is at the intersection of 6 roads is not reflected in the data. It is clear we cant look only at traffic flows per road, but also the intersection between roads.

In [ ]:
plt.rcParams['figure.dpi'] = 300

ax = ULEZ.plot(figsize=(20, 20), edgecolor='k', facecolor='none', linewidth=2, zorder = 1)

roads[(roads['DESC_TERM'] == "A Road") & ((roads['VKMTOTAL16'] > 100000) & (roads['VKMTOTAL16'] < 1000000))].plot(ax=ax, 
                                                                               column='VKMTOTAL16', 
                                                                               legend=False, 
                                                                               cmap='Reds', 
                                                                               linewidth = 3, zorder = 5)

gdf.plot(ax=ax, markersize = gdf['tot_avg_pm25']**3, zorder = 50, alpha=0.5)
gdf.plot(ax=ax, markersize = 20, zorder = 50, facecolor='black')

ax.set(xlim=(-20000, -5800), ylim=(6706000, 6717500))
ctx.add_basemap(ax=ax, url=ctx.providers.Stamen.TonerLite)

circ1 = Line2D([0], [0], linestyle="none", marker="o", markersize=25, markerfacecolor="lightgray", alpha = 0.85)
circ2 = Line2D([0], [0], linestyle="none", marker="o", markersize=12, markerfacecolor="lightgray", alpha = 0.85)

leg = ax.legend((circ1, circ2), ("Higher Average", "Lower Average"), 
                      numpoints=1, 
                      loc=4,
                      fontsize ='x-large')

ax.set_title('Relative A-Road traffic flow intensity and relative average PM2.5 readings for ULEZ area\n', 
                 fontdict={'fontsize': '20', 'fontweight' : '2'})

plt.axis('off')
plt.show()

In [ ]:
pm_cor = df[['tot_avg_pm25', 'motorway_min_dist', 'a_road_min_dist']].dropna()

### Minimum distances from road calculated in road_network script

### Correlation matrix for pm25 and minimum sensor distance from motorways or A-roads

In [ ]:
## Plot correlation matrix for pm25
cmap = sns.diverging_palette(220, 20, sep=20, as_cmap=True)
sns.clustermap(pm_cor.corr(), figsize= (20,20), cmap = cmap, annot=True).ax_row_dendrogram.set_visible(False)
plt.show()

### We observe no correlation between average sensor readings and distance from roads for pm 2.5

In [ ]:
no2_cor = df[['tot_avg_no2', 'motorway_min_dist', 'a_road_min_dist']].dropna()

In [ ]:
## Plot correlation matrix for pm25
cmap = sns.diverging_palette(220, 20, sep=20, as_cmap=True)
sns.clustermap(no2_cor.corr(), figsize= (20,20), cmap = cmap, annot=True).ax_row_dendrogram.set_visible(False)
plt.show()

### We observe a negative correlation of approximately 0.3 for No2 and distance from A-roads

### The below analysis would be greatly improved by using dynamic data. In the absence of that, this only looks at borough level data from London Borough Atlas 2016

In [ ]:
no2_traffic_cor = df[['no2_ugm3', ' VKM Motorcycle', ' VKM Petrol Car', ' VKM Diesel Car', ' VKM Taxi', ' VKM Electric Car', ' VKM Petrol LGV', ' VKM Diesel LGV', ' VKM Electric LGV', ' VKM Bus', ' VKM Coach', ' VKM Rigid HGV', ' VKM Artic HGV', ' VKM TOTAL']].dropna()

In [ ]:
## Plot correlation matrix for no2 traffic
cmap = sns.diverging_palette(220, 20, sep=20, as_cmap=True)
sns.clustermap(no2_traffic_cor.corr(), figsize= (20,20), cmap = cmap, annot=True).ax_row_dendrogram.set_visible(False)

plt.show()

### For No2 we observe a positive correlation with taxis, busses, coaches and motorcycles. There is no significant correlation between vehicle types and PM 2.5 concentrations 

### The plot below highlights the relationship between No2 concentrations and minimum main road distance. A stronger trend can be seen within a radius below 400 metres. 

In [ ]:
plt.rcParams['figure.figsize'] = (15,8)
plt.rcParams['figure.dpi'] = 300


fig, ax = plt.subplots()

sns.scatterplot(x=np.log(df["a_road_min_dist"]), 
                y=np.log(df["tot_avg_no2"]),
                 data=df,
                    hue = 'Site_Type_x',
                    ax=ax)

sns.regplot(x=np.log(df["a_road_min_dist"]), 
                 y=np.log(df["tot_avg_no2"]), 
                 data=df,
                    scatter = False,
                    ax=ax)


ax.set(xlabel='Sensor minimum distance from A-Road, Natural Log Scale', ylabel='Average No2 Levels in µg/m3, Natural Log Scale')
ax.set_title("Regression plot to show relationship between average No2 levels and distance from main road\n",
             fontdict={'fontsize': '16', 'fontweight' : '5'})

ax.text(0.37,2.6, "Pearson's correlation coefficient = -0.28\np-value = 0.00357")

legend = ax.legend(loc ='lower left')
legend.texts[0].set_text("Site Type")

plt.show()

In [ ]:
#pearsons correlation
p = df[['tot_avg_no2', 'a_road_min_dist']]
p = p.dropna(subset=['tot_avg_no2', 'a_road_min_dist'])

pearson = pearsonr(p['tot_avg_no2'], p['a_road_min_dist'])
pearson